In [ ]:
'''
Author: zhengyi cui
Date: 2024-10-25 22:09:44
E-mail address: Zhengyi.Cui@uth.tmc.edu
Copyright (c) 2024 by ${Zhengyi.Cui@uth.tmc.edu}, All Rights Reserved. 
'''



## Ordinary Kriging Interpolation for Spatial Data

This code performs spatial interpolation on PM2.5 data using Ordinary Kriging, a geostatistical method suitable for estimating values at unobserved locations. The code reads in PM2.5 data from a CSV file, creates an interpolation grid within a specified bounding box, and applies Ordinary Kriging for each unique date in the data. This can be useful for analyzing air quality variations over a defined geographic area.

### Key Components

- **Parameters**:
  - `data_dir`: Path to the CSV file (`./ground_pm25.csv`) containing PM2.5 data with columns for latitude (`lat`), longitude (`lon`), date, and the variable(s) to interpolate.
  - `variables`: A list of variables to be interpolated (e.g., `['value']`).
  - `lon_min`, `lon_max`, `lat_min`, `lat_max`: The geographic bounding box for the interpolation grid.
  - `grid_spacing`: Grid spacing interval for longitude and latitude (in degrees) used to define the density of interpolation points.

- **Output**: 
  - Returns a DataFrame `df_final` containing interpolated values for each grid point and date.


In [18]:
import pandas as pd
import numpy as np
from pykrige.ok import OrdinaryKriging

df_dir = './ground_pm25.csv'
lon_min = -108
lon_max = -105
lat_min = 30
lat_max = 33
grid_spacing = 0.1 #lon and lat grid spacing(intervals)
variables = ['value']

def ordinary_kriging(data_dir, variables, lon_min, lon_max, lat_min, lat_max, grid_spacing=grid_spacing, method='ordinary'):
    df = pd.read_csv(data_dir)
    unique_dates = df['date'].unique()
    interpolated_data = []
    for current_date in unique_dates:
        print(f"Processing data for {current_date}...")
        df_date = df[df['date'] == current_date]
        sample_lon = df_date['lon'].values
        sample_lat = df_date['lat'].values

        grid_lon = np.arange(lon_min, lon_max, grid_spacing)
        grid_lat = np.arange(lat_min, lat_max, grid_spacing)
        grid_lon_mesh, grid_lat_mesh = np.meshgrid(grid_lon, grid_lat)
        grid_lon_flat = grid_lon_mesh.ravel()
        grid_lat_flat = grid_lat_mesh.ravel()
        df_ok = pd.DataFrame({
            'date': pd.to_datetime(current_date),
            'lon': grid_lon_flat,
            'lat': grid_lat_flat
        })
        
        for var in variables:
            var_values = df_date[var].values
            if method == 'ordinary':
                OK = OrdinaryKriging(sample_lon, sample_lat, var_values, variogram_model='linear', verbose=False, enable_plotting=False)
                z_krige, ss = OK.execute('grid', grid_lon, grid_lat)

            z_flat = z_krige.ravel()
            df_ok[var] = z_flat
            print(f'Shape of Ordinary Kriging value: {z_flat.shape}')
        interpolated_data.append(df_ok)
    if interpolated_data:
        df_final = pd.concat(interpolated_data, ignore_index=True)
    return df_final

df_OK = ordinary_kriging(df_dir, variables, lon_min, lon_max, lat_min, lat_max, grid_spacing=grid_spacing, method='ordinary')

df = pd.read_csv(df_dir)
print(f"mean value before interpolation: {df['value'].mean()}, mean value after interpolation: {df_OK['value'].mean()}, \nstd value before interpolation: {df['value'].std()}, std value after interpolation: {df_OK['value'].std()}")
print(f'length of original data: {len(df)}, length of interpolated data: {len(df_OK)}')

Processing data for 2024-01-01...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-02...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-03...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-04...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-05...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-06...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-07...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-08...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-09...
Shape of Ordinary Kriging value: (900,)
Processing data for 2024-01-10...
Shape of Ordinary Kriging value: (900,)
mean value before interpolation: 7.635072611122669, mean value after interpolation: 7.433406550674059, 
std value before interpolation: 13.412809384574597, std value after interpolation: 10.406944380153638
length of original data: 40, length of interpolated da

## Regression Kriging

This code performs **regression kriging** to interpolate PM2.5 data, combining linear regression for trend estimation with kriging for residual spatial correlation.

### Key Steps
1. **Trend Estimation**: Fits a linear regression model to estimate spatial trends based on latitude and longitude.
2. **Residual Interpolation**: Applies ordinary kriging on residuals to account for spatial variability.



In [19]:
from sklearn.linear_model import LinearRegression

def regression_kriging(data_dir, lon_min, lon_max, lat_min, lat_max, grid_spacing=grid_spacing):
    df = pd.read_csv(data_dir)
    unique_dates = df['date'].unique()
    interpolated_data = []
    for current_date in unique_dates:
        print(f"Processing data for {current_date}...")
        df_date = df[df['date'] == current_date]
        sample_lon = df_date['lon'].values.reshape(-1, 1)
        sample_lat = df_date['lat'].values.reshape(-1, 1)
        pm25 = df_date['value'].values
        X = np.hstack((sample_lon, sample_lat))

        reg = LinearRegression()
        reg.fit(X, pm25)

        trend = reg.predict(X)

        residuals = pm25 - trend

        grid_lon = np.arange(lon_min, lon_max, grid_spacing)
        grid_lat = np.arange(lat_min, lat_max, grid_spacing)
        grid_lon_mesh, grid_lat_mesh = np.meshgrid(grid_lon, grid_lat)

        OK_residual = OrdinaryKriging(df_date['lon'].values, df_date['lat'].values, residuals, variogram_model='linear',
                                    verbose=False, enable_plotting=False)

      
        z_residual, ss = OK_residual.execute('grid', grid_lon, grid_lat)
        grid_lon_flat = grid_lon_mesh.ravel()
        grid_lat_flat = grid_lat_mesh.ravel()
        X_grid = np.vstack((grid_lon_flat, grid_lat_flat)).T
        trend_grid = reg.predict(X_grid)
        pm25_interpolated = trend_grid + z_residual.ravel()

        df_rk = pd.DataFrame({
            'date':pd.to_datetime(current_date),
            'lon': grid_lon_flat,
            'lat': grid_lat_flat,
            'value': pm25_interpolated
        })
        interpolated_data.append(df_rk)
        print('Shape of Regression Kriging value:', pm25_interpolated.shape)
    if interpolated_data:
        df_final = pd.concat(interpolated_data,ignore_index=True)
    else:
        print('No data to merge')
    
    return df_final

df_RK = regression_kriging(df_dir, lon_min, lon_max, lat_min, lat_max, grid_spacing=grid_spacing)
print(f"mean value before interpolation: {df['value'].mean()}, mean value after interpolation: {df_RK['value'].mean()}, \nstd value before interpolation: {df['value'].std()}, std value after interpolation: {df_RK['value'].std()}")
print(f'length of original data: {len(df)}, length of interpolated data: {len(df_RK)}')


Processing data for 2024-01-01...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-02...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-03...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-04...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-05...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-06...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-07...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-08...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-09...
Shape of Regression Kriging value: (900,)
Processing data for 2024-01-10...
Shape of Regression Kriging value: (900,)
mean value before interpolation: 7.635072611122669, mean value after interpolation: 32.96915021651383, 
std value before interpolation: 13.412809384574597, std value after interpolation: 108.69277420325453
length of original data: 40, lengt